<a href="https://colab.research.google.com/github/namsalmaongoroeva/test1/blob/main/04_Homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Задачи к Лекции 4

__Исходные данные__

Дан файл **"mlbootcamp5_train.csv"**. В нем содержатся данные об опросе 70000 пациентов с целью определения наличия заболеваний сердечно-сосудистой системы (ССЗ). Данные в файле промаркированы и если у человека имееются ССЗ, то значение **cardio** будет равно 1, в противном случае - 0. Описание и значения полей представлены во второй лекции.

__Загрузка файла__

In [12]:
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score, roc_auc_score, log_loss
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, log_loss
from matplotlib import pyplot as plt
from collections import Counter
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10, 5]


df = pd.read_csv("/content/mlbootcamp5_train.csv",
                 sep=";",
                 index_col="id")
# Делаем one-hot кодирование
chol = pd.get_dummies(df["cholesterol"], prefix="chol")
gluc = pd.get_dummies(df["gluc"], prefix="gluc")
df = pd.concat([df, chol, gluc], axis=1)

# Делаем пол бинарным признаком
df["gender_bin"] = df["gender"].map({1: 0, 2: 1})
df.head()

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio,chol_1,chol_2,chol_3,gluc_1,gluc_2,gluc_3,gender_bin
id,,,,,,,,,,,,,,,,,,,
0,18393,2,168,62.0,110,80,1,1,0,0,1,0,True,False,False,True,False,False,1
1,20228,1,156,85.0,140,90,3,1,0,0,1,1,False,False,True,True,False,False,0
2,18857,1,165,64.0,130,70,3,1,0,0,0,1,False,False,True,True,False,False,0
3,17623,2,169,82.0,150,100,1,1,0,0,1,1,True,False,False,True,False,False,1
4,17474,1,156,56.0,100,60,1,1,0,0,0,0,True,False,False,True,False,False,0


## Задачи

__1. Хоть в sklearn и присутствует реализация метода k-ближайших соседей, я же предлагаю попробовать вам написать его самостоятельно.__

* __создать классификатор используя только pandas, numpy и scipy. Гиперпараметром данного классификатора должно быть число ближайших соседей. (Необязательно) можно добавить метрику расстояния и выбор весов.__
* __С помощью кросс-валидации найти оптимальное количество ближайших соседей и (необязательно) набор признаков.__

Алгоритм работы классификатора:
 1. Для заданного прецедент  $\vec{x}$ мы считаем расстояние до всех прецедентов в обучающей выборке.
 2. Сортируем прецеденты по расстоянию до $\vec{x}$.
 3. Отбираем $k$ минимальных значений
 4. Устраиваем голосование между отобранными прецедент.

In [13]:
# A lot of code here
# Реализация метода k-ближайших соседей на numpy+pandas
class SimpleKNNClassifier:
    def __init__(self, n_neighbors=5):
        self.n_neighbors = n_neighbors

    def fit(self, X, y):
        self.X_train = np.array(X)
        self.y_train = np.array(y)

    def predict(self, X):
        X = np.array(X)
        y_pred = []
        for i in range(X.shape[0]):
            # Вычисляем евклидово расстояние
            distances = np.sqrt(np.sum((self.X_train - X[i, :]) ** 2, axis=1))
            # Находим индексы k-ближайших соседей
            nn_idx = np.argsort(distances)[:self.n_neighbors]
            # Голосование по меткам классов
            labels = self.y_train[nn_idx]
            majority = Counter(labels).most_common(1)[0][0]
            y_pred.append(majority)
        return np.array(y_pred)

In [14]:
# Кросс-валидация для выбора k
def cross_val_score_knn(X, y, k_values, n_splits=3):
    result = []
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    for k in k_values:
        acc_scores = []
        for train_idx, val_idx in kf.split(X):
            knn = SimpleKNNClassifier(n_neighbors=k)
            knn.fit(X.iloc[train_idx], y.iloc[train_idx])
            y_pred = knn.predict(X.iloc[val_idx])
            acc = accuracy_score(y.iloc[val_idx], y_pred)
            acc_scores.append(acc)
        mean_acc = np.mean(acc_scores)
        result.append((k, mean_acc))
        print(f"k={k}, acc={mean_acc}")
    return result

In [ ]:
# Выбор признаков
features = ['age', 'weight', 'height', 'ap_lo', 'ap_hi']
X = df[features]
y = df['cardio']

k_values = [1, 3, 5, 7, 10]
results = cross_val_score_knn(X, y, k_values)
best_k = max(results, key=lambda x: x[1])[0]
print("Лучший k:", best_k)

**Комментарии:** Ваши комментарии здесь.

**2. Определить какой из трех классификаторов (kNN, наивный Байес, решающее дерево) лучший в каждой метрике по отдельности: accuracy, F1-мера, ROC AUC, функция потерь. Использовать набор признаков: 'age', 'weight', 'height', 'ap_lo', 'ap_hi'.**

**(Необязательно) Найти оптимальный набор признаков.**

In [15]:
# Your code here
features_nb_tree = ['age', 'weight', 'height', 'ap_lo', 'ap_hi']
X = df[features_nb_tree]
y = df['cardio']

# Разделим на тестовую выборки:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.25, random_state=42, stratify=y)

# kNN
knn = SimpleKNNClassifier(n_neighbors=best_k)
knn.fit(X_train, y_train)
y_pred_knn = knn.predict(X_test)

# Наивный Байес
nb = GaussianNB()
nb.fit(X_train, y_train)
y_pred_nb = nb.predict(X_test)
y_proba_nb = nb.predict_proba(X_test)

# Дерево решений
dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_train, y_train)
y_pred_dt = dt.predict(X_test)
y_proba_dt = dt.predict_proba(X_test)

In [16]:
# Сравнение по метрикам
def evaluate(y_true, y_pred, y_proba=None):
    acc = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    auc = roc_auc_score(y_true, y_proba[:,1]) if y_proba is not None else None
    ll = log_loss(y_true, y_proba) if y_proba is not None else None
    return acc, f1, auc, ll

# Для kNN вероятность считать как долю среди ближайших, но реализуем только базовые метрики
acc_knn = accuracy_score(y_test, y_pred_knn)
f1_knn = f1_score(y_test, y_pred_knn)

acc_nb, f1_nb, auc_nb, ll_nb = evaluate(y_test, y_pred_nb, y_proba_nb)
acc_dt, f1_dt, auc_dt, ll_dt = evaluate(y_test, y_pred_dt, y_proba_dt)

print(f"kNN:      acc={acc_knn:.3f}, f1={f1_knn:.3f}")
print(f"Bayes:    acc={acc_nb:.3f}, f1={f1_nb:.3f}, auc={auc_nb:.3f}, logloss={ll_nb:.3f}")
print(f"Tree:     acc={acc_dt:.3f}, f1={f1_dt:.3f}, auc={auc_dt:.3f}, logloss={ll_dt:.3f}")

kNN:      acc=0.692, f1=0.684
Bayes:    acc=0.547, f1=0.243, auc=0.668, logloss=0.758
Tree:     acc=0.615, f1=0.616, auc=0.615, logloss=13.858


**Комментарии:** Ваши комментарии здесь.